In [ ]:
import os
import tempfile
import mappyfile
import copy
import geopandas as gpd
from sqlalchemy import create_engine
import subprocess
from PIL import Image, ImageOps

Read the configuration file to get parameters

In [ ]:
from configparser import ConfigParser
config = ConfigParser()
config.read("setting.ini")
dbsett = config["eurodeer_db"]
eu_bbox = [-12, 35, 34, 65]

In [ ]:
def create_mapfile(network, bounds, dbsett, dbname, table="main.research_group", idcol="research_groups_id", template="euromammals_template.map"):
    """This function is creating the final mapfile using a template"""
    mapfile = mappyfile.open(template)
    mapfile['name'] = mapfile['name'].format(network.upper())
    if bounds != None and template != "world_template.map":
        mapfile["extent"] = list(bounds)
    layers = mapfile['layers']
    removenumber = None
    redlayer = None
    x = 0
    for layer in layers:
        if layer["name"] == "Study areas":
            if network == "eurodeer":
                redlayer = copy.deepcopy(layer)
                layer["name"] = "EURODEER study areas"
                layer["classes"][0]["name"] = "EURODEER study areas"
            layer["classes"][0]["styles"][0]["symbol"] = layer["classes"][0]["styles"][0]["symbol"].format(network=network)
            if network == 'eurolynx':
                removenumber = x
        if layer["name"] == "Reserch groups":
            table = "main.research_groups"
            if network == "euromammals":
                table = "main.view_research_groups_euromammals_cam"
                idcol = "id"
            if network == "eurocam":
                idcol = "research_group_code"
            strbound = ','.join(str(i) for i in bounds)
            layer["data"][0] = layer["data"][0].format(bound=strbound, table=table, id=idcol)
        if "connectiontype" in layer.keys():
            if layer["connectiontype"].upper() == 'POSTGIS':
                layer["connection"] = layer['connection'].format(host=dbsett["host"], user=dbsett["user"], password=dbsett["password"], network=dbname)
        x += 1
    if network == "eurolynx" and removenumber is not None:
        layers.pop(removenumber)
        mapfile['layers'] = layers
    if redlayer is not None:
        redlayer["name"] = "EUREDDEER study areas"
        redlayer["classes"][0]["name"] = "EUREDDEER study areas"
        redlayer["classes"][0]["styles"][0]["symbol"] = redlayer["classes"][0]["styles"][0]["symbol"].format(network="eureddeer")
        redlayer["connection"] = redlayer['connection'].format(host=dbsett["host"], user=dbsett["user"], password=dbsett["password"], network="eureddeer_db")
        mapfile['layers'].insert(1, redlayer)
    validation = mappyfile.validate(mapfile)
    if len(validation) == 0:
        return mapfile
    else:
        print("Errors during mapfile validation:")
        print("\n".join(validation))
        return None

In [ ]:
def bigger_bbox(bbox1, bbox2):
    """Return the right bigger bounding box"""
    return (min(bbox1[0], bbox2[0]), min(bbox1[1], bbox2[1]), max(bbox1[2], bbox2[2]), max(bbox1[3], bbox2[3]))

In [ ]:
def grow_bbox(bbox1, value=2):
    """Grow the box of a some value"""
    return (bbox1[0] - value, bbox1[1] - value, bbox1[2] + value, bbox1[3] + value)

Choose the network to use

In [ ]:
networks = ["eurodeer", "euroboar", "eurowildcat", "euroibex", "eurolynx", "eurosmallmammals", "eurocam", "euromammals"]

In [ ]:
network = networks[7]
if network == "eurosmallmammals":
    noneugroups_df = []

Get the data for the selected network

In [ ]:
dbname = "{net}_db".format(net=network)
if network == "euromammals":
    dbname = "eurodeer_db"
db_connection_url = "postgresql://{us}:{pas}@{host}:{port}/{db}".format(us=dbsett['user'],
                                                                      pas=dbsett['password'],
                                                                      host=dbsett['host'],
                                                                      port=dbsett['port'],
                                                                      db=dbname
                                                                     )
con = create_engine(db_connection_url)
if network != "eurolynx":
    sid = "study_areas_id"
    sname = "study_name"
    geom = "geom"
    if network == "eurocam":
        sid = "study_area_code"
        sname = "study_area_name"
        geom = "geom_centroid"
    query_areas = f"select {sid} as id, {sname} as name, {geom} as geom from main.study_areas where geom is not null"
    areas_df = gpd.GeoDataFrame.from_postgis(query_areas, con)
else:
    areas_df = []
rid = "research_groups_id"
geom = "geom"
if network == "eurocam":
    rid = "research_group_code"
bbox = ",".join(str(n) for n in eu_bbox)
table = "main.research_groups"
if network == "euromammals":
    table = "main.view_research_groups_euromammals_cam"
    rid = "id"
query_eugroups = f"select {rid} as id, short_name as name, {geom} from {table} where geom @ ST_MakeEnvelope ({bbox}, 4326) order by st_y(geom)"
eugroups_df = gpd.GeoDataFrame.from_postgis(query_eugroups, con)
query_noneugroups = f"select {rid} as id, short_name as name, {geom} from {table} where not ST_Contains(ST_MakeEnvelope ({bbox}, 4326), geom) order by st_y(geom)"
noneugroups_df = gpd.GeoDataFrame.from_postgis(query_noneugroups, con)

In [ ]:
print(query_eugroups)

Select the maximum extend

In [ ]:
areas_bounds = []
if len(areas_df) > 0:
    areas_bounds = grow_bbox(areas_df.geom.unary_union.bounds)
groups_bounds = grow_bbox(eugroups_df.geom.unary_union.bounds)

In [ ]:
if areas_bounds:
    newbounds = bigger_bbox(areas_bounds, groups_bounds)
else:
    newbounds = groups_bounds
print(newbounds)

Create the mapfile for the current network

In [ ]:
network_mapfile = create_mapfile(network, newbounds, dbsett, dbname)#, "euromammals_template_areaname.map")
if network_mapfile:
    mappyfile.save(network_mapfile, "{}.map".format(network))

Call `shp2img` command to create the map

In [ ]:
shp2img_call = "shp2img -m {path}.map -o {path}.png".format(path=network)
print(shp2img_call)
shp2img = subprocess.run(shp2img_call, shell=True, capture_output=True)
print(shp2img.stderr)
print(shp2img.stdout)

Create the legend

In [ ]:
legend_call = "legend {path}.map {path}_legend.png".format(path=network)
legend = subprocess.run(legend_call, shell=True, capture_output=True)
print(legend.stderr)
print(legend.stdout)

Create the image joining map and legend, if there are institute outside Europe it calculate also the world map

In [ ]:
mapimg = Image.open("{path}.png".format(path=network))
legimg = Image.open("{path}_legend.png".format(path=network))
legimg_white = ImageOps.expand(legimg, border=2, fill='white')
legimg_border = ImageOps.expand(legimg_white, border=1)
if network != "eurocam":
    logo = Image.open("logos/{path}_logo.png".format(path=network))
    logo.load()
    newlogo = Image.new("RGBA", logo.size, (255, 255, 255, 255))
    newlogo.paste(logo, mask=logo.split()[3])
    mapimg.paste(newlogo, (20, 30))
mapimg.paste(legimg_border, (20, 150))
if network == 'eurodeer':
    redlogo = Image.open("logos/eureddeer_logo.png")
    redlogo.load()
    newredlogo = Image.new("RGB", redlogo.size, (255, 255, 255))
    newredlogo.paste(redlogo, mask=redlogo.split()[3])
    mapimg.paste(newredlogo, (newlogo.width + 50, 15))
if len(noneugroups_df) > 0:
    world_mapfile = create_mapfile(network, newbounds, dbsett, dbname, template="world_template.map")#, "euromammals_template_areaname.map")
    if world_mapfile:
        mappyfile.save(world_mapfile, "world_{}.map".format(network))
        shp2img_worldcall = "shp2img -m world_{path}.map -o world_{path}.png".format(path=network)
        shp2img_world = subprocess.run(shp2img_worldcall, shell=True, capture_output=True)
        print(shp2img_world.stdout)
        print(shp2img_world.stderr)
        if not shp2img_world.stderr:
            worldimg = Image.open("world_{path}.png".format(path=network))
            worldimg_border = ImageOps.expand(worldimg, border=1)
            mapimg.paste(worldimg_border, (mapimg.width - worldimg_border.width - 10, 25))
        
mapimg.save("{path}_final.png".format(path=network))